In [ ]:
import requests
import json
import pandas as pd
from typing import Any, List, Mapping, Optional
from pandas import DataFrame

In [ ]:
def get_sentence(mention:Mapping[str, Any], documents:Mapping[str, Any]):
	""" Returns the text containing the mention """
	doc_id = mention['document']
	sentence_ix = mention['sentence']

	sentence = documents[doc_id]['sentences'][sentence_ix]

	return ' '.join(sentence['words'])


def build_groundings(mention:Mapping[str, Any]) -> Optional[str]:
	""" Extracts the grounding ID of the grounding associated to the current mention """

	
	if mention['type'] == "TextBoundMention":
		groundings = list()
		for a in mention['attachments']:
			# If score is a field in the attachment, then this is a grounding
			if type(a) == list:
				for g in a[0]:
					score = "{:.2f}".format(g['score'])
					groundings.append(f"({g['name']}, {g['id']}, {score})")
		if len(groundings) > 0:
			return ', '.join(groundings)
			
	

In [ ]:
def annotate_json(file_path:str, endpoint:str = "http://localhost:9000/cosmos_json_to_mentions") -> List[Mapping[str, Any]]:
	""" Annotates an existing json file on the server with the text reading pipeline """

	data = requests.post(endpoint, json={'pathToCosmosJson': file_path}).json()

	return data


def build_data_frame(mentions:List[Mapping[str, Any]], documents:Mapping[str, Any]) -> DataFrame:
	""" Builds a data frame from the output of text reading """

	return DataFrame(
			{
				'text': mention['text'],
				'sentence': get_sentence(mention, documents),
				'start_token': mention['tokenInterval']['start'],
				'end_token': mention['tokenInterval']['end'],
				'mention_type': mention['type'],
				'label': mention['labels'][0],
				'grounding_id': build_groundings(mention),
				# 'sentence_ix': mention['sentence'],				
			}
			for mention in mentions
		)

def text_reading_mentions(file_path:str) -> DataFrame:
	""" Puts together annotation and creating the data frame"""
	
	return build_data_frame(**annotate_json(file_path))

In [ ]:
def get_paper_by_doi(doi:str) -> Optional[str]:
	""" Mocks getting the Cosmos output by the paper's DOI """
	papers = {
		"10.1016/j.chaos.2021.110689": "/media/evo870/github/COSMOS-data/output_files/documents_5Febcovid19--COSMOS-data.json"
	}

	return papers.get(doi)

In [ ]:
pd.set_option('display.max_colwidth', None)

paper_path = get_paper_by_doi("10.1016/j.chaos.2021.110689")

text_reading_mentions(paper_path)